In [ ]:
import numpy as np

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time
from datetime import datetime, timezone
from astroplan import Observer, FixedTarget
from astroplan.plots import plot_airmass, plot_sky

import matplotlib.pyplot as plt
#%matplotlib widget

In [ ]:
#time = Time('2024-11-15 03:00:00.000', format='iso')
time = Time(datetime.now(timezone.utc))
observer = Observer.at_site("LSST")

ra = 53.125
dec = -28.1
name = 'ECDFS'
coord = SkyCoord(ra=ra, dec=dec, unit='deg', frame='icrs')

In [ ]:
time_midnight = observer.midnight(time, which='nearest')
time_sunset = observer.sun_set_time(time_midnight, which='previous')
time_sunrise = observer.sun_rise_time(time_midnight, which='next')

if observer.moon_altaz(time_midnight).alt.value > 0.:
    time_moonrise = observer.moon_rise_time(time_midnight, which='previous')
    time_moonset = observer.moon_set_time(time_midnight, which='next')
else:
    time_moonrise = observer.moon_rise_time(time_midnight, which='next')
    time_moonset = observer.moon_set_time(time_midnight, which='previous')

print('Time (UTC) =', time)
print('Time Midnight (UTC) =', time_midnight)
print('Time Evening Civil Twilight (UTC) =', observer.twilight_evening_civil(time_midnight, which='previous').iso)
print('Time Evening Nautical Twilight (UTC) =', observer.twilight_evening_nautical(time_midnight, which='previous').iso)
print('Time Evening Astronomical Twilight (UTC) =', observer.twilight_evening_astronomical(time_midnight, which='previous').iso)
print('Time Morning Astronomical Twilight (UTC) =', observer.twilight_morning_astronomical(time_midnight, which='next').iso)
print('Time Evening Nautical Twilight (UTC) =', observer.twilight_morning_nautical(time_midnight, which='previous').iso)
print('Time Evening Civil Twilight (UTC) =', observer.twilight_morning_civil(time_midnight, which='previous').iso)
print('Target = %s'%(name))
print('(RA, Dec) = (%.2f, %2f)'%(ra, dec))
print('Target Az (deg) =', observer.altaz(time, coord).az.deg)
print('Target Alt (deg) =', observer.altaz(time, coord).alt.deg)
print('Target Transit (UTC) =', observer.target_meridian_transit_time(time_midnight, coord, which='nearest').iso)
print('Moon Illumination =', observer.moon_illumination(time))
print('Moonrise time (UTC) =', time_moonrise.iso)
print('Moonset time (UTC) =', time_moonset.iso)
print('Lunar Separation (deg) =', observer.moon_altaz(time).separation(observer.altaz(time, coord)).deg)

In [ ]:
time_array = time_sunset + np.arange(0., (time_sunrise - time_sunset).to(u.hr).value, 0.1) * u.hr

alt_array = observer.altaz(time_array, coord).alt.deg
az_array = observer.altaz(time_array, coord).az.deg

print('%30s%15s%15s'%('Time (UTC)', 'Alt (deg)', 'Az (deg)'))
for ii in range(0, len(time_array)):
    print('%30s%15.2f%15.2f'%(time_array[ii].iso, alt_array[ii], az_array[ii]))

In [ ]:
target = FixedTarget(coord, name=name)

plt.figure()
plt.clf()
ax = plot_airmass(target, observer, time_midnight, brightness_shading=True, altitude_yaxis=True)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

In [ ]:
target_moon = FixedTarget(observer.moon_altaz(time), name='Moon')

plt.clf()
plot_sky(target, observer, time_array)
plot_sky(target, observer, time)
plot_sky(target_moon, observer, time_array)
plot_sky(target_moon, observer, time)
plt.legend(loc='lower right')
#bbox_to_anchor=(1.25, 0.5))
plt.show()